## M0 : Quick Start track B

### Confirm GPU is ready

In [1]:
!nvidia-smi || echo "nvidia-smi unavailable (CPU runtime)"

Tue Nov 25 19:57:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 577.03                 Driver Version: 577.03         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   61C    P0             27W /   50W |    2578MiB /   8188MiB |     59%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Point the notebook at the project folder

In [2]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent.resolve()
elif PROJECT_ROOT.name == "content":
    candidate = PROJECT_ROOT / "od-project"
    if candidate.exists():
        PROJECT_ROOT = candidate.resolve()

if not (PROJECT_ROOT / "src").exists():
    raise FileNotFoundError(
        f"Could not locate project root at {PROJECT_ROOT}. Upload or clone od-project before proceeding."
    )

os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT / "src") not in sys.path:
    sys.path.append(str(PROJECT_ROOT / "src"))
print(f"Project root: {PROJECT_ROOT}")

Project root: C:\Users\lucas\End-to-End-Deep-Learning-Systems\End-to-End-Deep-Learning-Systems\starters\od-project-starter\od-project


### Install the project requirement

In [3]:
# Install project dependencies listed in requirements.txt
!pip install -r requirements.txt

### Run the smoke test

In [10]:
from src import smoke_check

smoke_path = smoke_check.run_smoke("configs/yolo_coco128.yaml")
print(smoke_path.read_text())

Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
{
  "num_boxes": 6,
  "mean_confidence": 0.6556,
  "imgsz": 640,
  "weights": "yolov8n.pt"
}


# Track B Object Detection **yolo_coco128**

## M1: Problem Scoping & Data Validation

### 1.1. Problem Definition

#### **Business Case**

Dans ce Track B, notre objectif est de **prototyper un système de perception pour un véhicule autonome ou un drone de gestion d’inventaire**.
Plus précisément, nous devons :

* détecter plusieurs objets dans une image,
* localiser leurs boîtes englobantes (bounding boxes),
* produire une estimation de classe + confiance,
* évaluer la qualité du modèle via des métriques adaptées à la détection.

#### **Task Definition**

* **Type de tâche :** *Object Detection*
* **Famille de modèles :** YOLOv8 (Ultralytics)
* **Output modèle :**

  * Liste de boîtes (x1, y1, x2, y2)
  * Logits → probas de classes
  * Score de confiance pour chaque prédiction

#### **Pourquoi YOLO pour ce Track ?**

* Modèle moderne et optimisé GPU
* API simple (Ultralytics)
* Training + eval + inference en quelques lignes
* Fonctionne parfaitement sur un dataset léger comme **coco128**


### 1.2. Evaluation Metrics

Les métriques de référence pour la détection d’objets sont :

#### **mAP@0.5**

Mean Average Precision à IoU = 0.5
Demande que la prédiction chevauche l’objet avec 50% d’intersection / union.

#### **mAP@[0.5:0.95]**

Moyenne de l’AP pour des IoU allant de 0.5 à 0.95 par pas de 0.05
C’est **la métrique officielle COCO**
Beaucoup plus stricte et informative

#### **Precision & Recall**

* **Précision** : proportion de prédictions correctes
* **Recall** : proportion des objets trouvés
* **Trade-off** → important à analyser selon le use case (ex. véhicule autonome vs drone d’inventaire)

#### **Confusion matrix (adaptée detection)**

Ultralytics génère automatiquement :

* matrice des classes détectées vs réelles,
* histogrammes des scores,
* courbes PR,
* courbes F1.


### 1.3. Data Card – yolo_coco128

Dans notre projet, le dataset est référencé via la section `data` du YAML :

```yaml
data:
  yaml: coco128.yaml
  imgsz: 640
  batch: 16
```

* `yaml: coco128.yaml` pointe vers un fichier YOLO classique (data.yaml)
  qui définit les chemins des images et des labels (`train`, `val`, …).
* `imgsz: 640` impose une résolution d’entrée carrée (640×640) utilisée
  à la fois pour l’entraînement et la validation.
* `batch: 16` contrôle la taille des mini-batchs par itération.

Le fichier `coco128.yaml` contient :

* les chemins vers `train` et `val`,
* la liste des classes (80 classes COCO),
* le nombre de classes `nc`.


### 🔸 1.4 Train / Validation / Test splits (à partir de `coco128.yaml`)

Dans ton notebook tu peux ajouter une petite cellule pour **montrer réellement le split** :



### 1.4. Train / Validation / Test splits

Le dataset coco128 utilise la structure :

* **Train :** `images/train/` + `labels/train/`
* **Val :** `images/val/` + `labels/val/`
* **Test :** non fourni (classique sur les mini-datasets COCO)

Dans YOLO + Ultralytics, les splits sont définis dans le fichier YAML :

`configs/yolo_coco128.yaml`

In [13]:
from utils import load_yaml

cfg = load_yaml("configs/yolo_coco128.yaml")
data_cfg = load_yaml(cfg["data"]["yaml"])

data_cfg

FileNotFoundError: [Errno 2] No such file or directory: 'coco128.yaml'

Le fichier de config `configs/yolo_coco128.yaml` délègue la définition
des splits au fichier `coco128.yaml` :

- `train` : chemin vers les images/labels d’entraînement
- `val`   : chemin vers les images/labels de validation
- (pas de split `test` explicite pour coco128, ce qui est classique
  pour un mini-dataset de démonstration)

Notre pipeline est donc :

- Entraînement uniquement sur le split `train`
- Suivi de performances sur le split `val` via mAP@0.5, mAP@0.5:0.95,
  précision et rappel
- Sauvegarde des artefacts sous `outputs/` (défini par `output_dir`)
  avec les poids du meilleur modèle sous `outputs/train/weights/best.pt`.